In [ ]:
import pandas
import numpy
import matplotlib.pyplot
import datetime
%matplotlib inline
# Data files:
device = pandas.read_csv('device_info.csv')
email = pandas.read_csv('email_info.csv')
employee = pandas.read_csv('employee_info.csv')
http = pandas.read_csv('http_info.csv')
logon = pandas.read_csv('logon_info.csv')

Our first set of suspects we looked at were people that were fired and also at thier supervisors. We quickly saw that this on its own was not enough to figure out who to focus on. However we can compare future analytics against this list to help us validate our suspicions

In [ ]:
user_ids = numpy.unique(employee.user_id)
month = [""] * len(user_ids)
fired_date = pandas.DataFrame()
fired_date["user_id"] = user_ids
fired_date["month"] = month
fired_date = fired_date.set_index("user_id")

for i in range(0, len(employee)):
    currentRow = employee.iloc[i,:]
    fired_date.loc[currentRow.user_id].month = currentRow.month
    
people_fired = fired_date[fired_date.month != "11/1/17"].copy()

In [ ]:
names = []
supervisor = []
for i in people_fired.index.get_values():
    hold = employee[employee.user_id == i]
    names.append(hold.employee_name.iloc[0])
    supervisor.append(hold.supervisor.iloc[0])

people_fired["name"] = names
people_fired["supervisor"] = supervisor
people_fired

In [ ]:
supervisor_fire_count = pandas.DataFrame(people_fired.groupby("supervisor").count())
del supervisor_fire_count["name"]
supervisor_fire_count.rename(columns = {"month" : "Amount of people Fired"}, inplace=True)
supervisor_fire_count

The next set of data that we decided to analyze was activity that occured outside of the normal work day. And again by its self we could not see any clear path to continue our investigation from just this data but will be usefull in documenting all suspicious activities of the employes.

In [ ]:
strangeEarly = datetime.datetime.strptime("06:00:00", '%H:%M:%S')
strangeLate = datetime.datetime.strptime("18:00:00", '%H:%M:%S')

In [ ]:
email
emailOffHours = email
emailOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in emailOffHours.date]
emailOffHours = emailOffHours[emailOffHours.offHours]
emailOffHours["day"] = [x.split(" ")[0] for x in emailOffHours.date]
emailOffHours

In [ ]:
device
deviceOffHours = device
deviceOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in deviceOffHours.date]
deviceOffHours = deviceOffHours[deviceOffHours.offHours]
deviceOffHours["day"] = [x.split(" ")[0] for x in deviceOffHours.date]
deviceOffHours

In [ ]:
logon
logonOffHours = logon
logonOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in logonOffHours.date]
logonOffHours = logonOffHours[logonOffHours.offHours]
logonOffHours["day"] = [x.split(" ")[0] for x in logonOffHours.date]
logonOffHours

In [ ]:
http
httpOffHours = http
httpOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in httpOffHours.date]
httpOffHours = httpOffHours[httpOffHours.offHours]
httpOffHours["day"] = [x.split(" ")[0] for x in httpOffHours.date]
httpOffHours

In [ ]:
httpOffHours.iloc[:,:].groupby("user").day.count().plot.barh(figsize=(20,50))

Building off of the last idea we decided to look at every record and mark down if suspicious activites occured, for now it is just a 1 if it occured and a 0 if it did not occur but in the future we can change it to be < 1 if an event occured and the user's job make sense for them to be causing that event (e.x. itadmins will have smaller number for working outside of work hours). 

In [ ]:
susEmail = email.copy()
susEmail["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susEmail.date]
susEmail["outside"] = [(not("@dtaa.com" in x)) for x in susEmail.to]
eToU = dict(zip(employee.email, employee.user_id))
uToE = dict(zip(employee.user_id, employee.email))
susEmail["user"] = [eToU[x] for x in susEmail.iloc[:,3]]

In [ ]:
susHttp = http.copy()
susHttp["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susHttp.date]
susHttp["money"] = [ ("bank" in x) or ("pay" in x)  for x in susHttp.url]

In [ ]:
susLogon = logon.copy()
susLogon["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susLogon.date]

In [ ]:
susDevice = device.copy()
susDevice["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susDevice.date]

In [ ]:
susHttp["offHours"] = susHttp["offHours"].apply(float)
susHttp["money"] = susHttp["money"].apply(float)
susEmail["offHours"] = susEmail["offHours"].apply(float)
susEmail["outside"] = susEmail["outside"].apply(float)
susLogon["offHours"] = susLogon["offHours"].apply(float)
susDevice["offHours"] = susDevice["offHours"].apply(float)
susHttp["day"] = [x.split(" ")[0] for x in susHttp.date]
susEmail["day"] = [x.split(" ")[0] for x in susEmail.date]
susLogon["day"] = [x.split(" ")[0] for x in susLogon.date]
susDevice["day"] = [x.split(" ")[0] for x in susDevice.date]


Then we looked at the people with the most suspicious activity overall

In [ ]:
df = pandas.DataFrame(susLogon.groupby("user").offHours.sum())
df.rename(columns={"day": "logon"}, inplace=True)
df["http"] = susHttp.groupby("user").offHours.sum() + susHttp.groupby("user").money.sum()
df["email"] = susEmail.groupby("user").offHours.sum() + susEmail.groupby("user").outside.sum()
df["device"] = susDevice.groupby("user").offHours.sum()

df = df.fillna(0.0)
df["sum"] = df.sum(axis=1)
df = df.sort_values("sum", ascending=False)


Lastly for this analytics we decided to look at the top X suspicious people and look at thier mean suspicion for each day and the days prior. The idea here is to ignore people who are always supicious and to look for people who wernt suspicious who became suspicious then went back to not being suspicious, they might be involved in the incident and it could give us a timeframe for the incident also. 

In [ ]:
for suspect in df.index[0:20]:
    susEmail[susEmail.user == suspect].groupby("day").offHours.sum()
    suspect1 = pandas.DataFrame(susLogon[susLogon.user == suspect].groupby("day").offHours.sum())
    suspect1.rename(columns={"offHours": "logon"}, inplace=True)
    suspect1["http"] = susHttp[susHttp.user == suspect].groupby("day").offHours.sum() + susHttp[susHttp.user == suspect].groupby("day").money.sum()
    suspect1["email"] = susEmail[susEmail.user == suspect].groupby("day").offHours.sum() + susEmail[susEmail.user == suspect].groupby("day").outside.sum()
    suspect1["device"] = susDevice[susDevice.user == suspect].groupby("day").offHours.sum()

    suspect1 = suspect1.fillna(0.0)
    suspect1["total"] =suspect1.sum(axis=1)
    #suspect1 = suspect1.sort_values("sum", ascending=False)
    suspect1["consistency"] = [ suspect1.total.iloc[:i].mean()for i in range(len(suspect1))]
    suspect1.plot.line(x=suspect1.index, y="consistency", figsize=(15,5), title=suspect)